# BUILD LLM Bootcamp Day 2

### Install Libraries

In [25]:
!pip install --no-deps --force-reinstall snowflake_ml_python-1.0.12-py3-none-any.whl

Processing ./snowflake_ml_python-1.0.12-py3-none-any.whl
  Attempting uninstall: snowflake-ml-python
    Found existing installation: snowflake-ml-python 1.0.12
    Uninstalling snowflake-ml-python-1.0.12:
      Successfully uninstalled snowflake-ml-python-1.0.12


In [26]:
!pip install peft transformers==4.34.0 tokenizers vllm==0.2.1.post1 bitsandbytes datasets absl-py==1.3.0

In [27]:
# TODO: Set your Hugging Face token
!huggingface-cli login --token 

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Import Libraries

In [33]:
from snowflake.snowpark.session import Session
from snowflake.ml.model.models import llm
from snowflake.ml.registry import model_registry
from snowflake.ml.model import deploy_platforms
import snowflake.ml.playground.finetune as ft_util
from snowflake.snowpark import VERSION
import snowflake.snowpark.functions as F
from datasets import load_dataset, ClassLabel, Dataset

import sys
import os
import json
import pandas as pd
pd.set_option('display.max_colwidth', None)

import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)
logger = logging.getLogger("snowflake.ml")
logger.setLevel(logging.ERROR)

### Establish Secure Connection

*NOTE: Update [connection.json](connection.json) and set your password, Hugging Face token, and replace '####' with your user number.*

In [29]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : DASH
Role                        : "DASH_SPCS"
Database                    : "DASH_DB"
Schema                      : "DASH_SCHEMA"
Warehouse                   : "DASH_L"
Snowflake version           : 7.40.0
Snowpark for Python version : 1.9.0


### Fine-tune Llama 2 in Snowpark Container Services

#### Load Training Dataset

*NOTE: Fine-tuning using 100 records on this setup (compute and other resources) will take about ~5mins.*

In [34]:
def prepare_stratified_dataset(path, seed = 42):
    raw_df = pd.read_json(path, lines=True)
    raw_df['id'] = raw_df.index
    ds = Dataset.from_pandas(raw_df, split='train')
    cl = ClassLabel(num_classes=4, names=["EN", "FR", "DE", "ES"])
    new_features = ds.features.copy()
    new_features['lang_label'] = cl
    cl_d = {l : cl.str2int(l) for l in ["EN", "FR", "DE", "ES"]}
    def convert_lang(sample):
        sample['lang_label'] = cl_d[sample['language']]
        return sample
    ds = ds.map(convert_lang, features=new_features)
    ds_split = ds.train_test_split(test_size=0.15, stratify_by_column='lang_label', seed=42)
    test_ds_split = ds_split['test'].train_test_split(test_size=2/3, stratify_by_column='lang_label', seed=42)
    return ds_split['train'].to_pandas(), test_ds_split['train'].to_pandas(), test_ds_split['test'].to_pandas()

In [35]:
# df = pd.read_json("transcripts.json", lines=True)
# print(f"Total records: {df.shape[0]}")
# train_df = df.head(100)
# print(f"Train        : {train_df.shape[0]}")
# eval_df = df[200:300]
# print(f"Eval         : {eval_df.shape[0]}")
# test_df = df.tail(100)
# print(f"Test         : {test_df.shape[0]}")

In [36]:
train_df, eval_df, test_df = prepare_stratified_dataset(
    'transcripts.json'
)

Map:   0%|          | 0/1368 [00:00<?, ? examples/s]

In [37]:
train_df.head()

,instruction,input,output,language,id,lang_label
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Greetings! I'm Frosty, your voice bot assistant for the holiday season. What's your name?\ncaller: Hey Frosty, my name is James.\nfrosty: Hi James! Where are you calling from?\ncaller: I'm calling from Sydney.\nfrosty: Nice to meet you, James from Sydney! Let's get started with your holiday wish list. What toy would you like to choose?\ncaller: Hmm... I'm not sure yet. What do you recommend?\nfrosty: I'm happy to help you explore our catalog! Does your child like plush toys?\ncaller: Yes, my little one definitely loves plush toys.\nfrosty: We have a sweet 2023 Holiday Fox 12-Inch Plush, or how about the Sesame Street Monster Meditation Elmo? Both are soft and cuddly.\ncaller: Oh, Elmo for sure!\nfrosty: The Sesame Street Monster Meditation Elmo is an excellent choice. Tell me, what do you enjoy the most about this time of the year?\ncaller: I love seeing all the decorations and spending time with my family.\nfrosty: That's wonderful! Do you have any favorite holiday memories?\ncaller: One year, we had a fantastic snowball fight in the park with our friends. It was so much fun!\nfrosty: That sounds like a terrific memory! So, are we adding Sesame Street Monster Meditation Elmo to your wish list?\ncaller: Yes! And you know what? Add the fox plushie as well.\nfrosty: You got it, adding the 2023 Holiday Fox 12-Inch Plush and Sesame Street Monster Meditation Elmo to your wish list. Have a great holiday season, James!\ncaller: Thanks, Frosty! You too!""","{""toy_list"": [""2023 Holiday Fox 12-Inch Plush"", ""Sesame Street Monster Meditation Elmo""], ""location"": ""Sydney""}",EN,245,0
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi there!\nfrosty: Hello! Welcome to our holiday hotline, I'm Frosty. How can I assist you today?\ncaller: Hey, Frosty. My name is Mike, and I live in Los Angeles.\nfrosty: Hi, Mike from Los Angeles! What can I help you with?\ncaller: I want to submit my holiday wish list, but I'm not sure what toys my son would like.\nfrosty: No problem! I'd be happy to help you explore our catalog. What are some of his interests?\ncaller: He loves playing with new gadgets and is a fan of Star Wars.\nfrosty: How about Star Wars LOLA animatronic droid? It's a fun gadget for any Star Wars fan.\ncaller: That sounds interesting. What else do you have?\nfrosty: Another option could be a Dog-E, the robot dog. It's a popular gadget for kids this holiday season.\ncaller: Let's go with the lola droid. But I also need a second toy.\nfrosty: Alright! Can you tell me more about what he enjoys during this time of the year?\ncaller: He loves playing outdoors and making snow forts with his friends.\nfrosty: In that case, he might be interested in the Tiny Land kids fort building kit. It allows him to create forts both indoors and outdoors, perfect for all seasons.\ncaller: Great, add both the lola droid and fort building kit to our wish list, please.\nfrosty: Excellent choices, Mike! I'll add the Star Wars LOLA animatronic droid and the Tiny Land kids fort building kit to your wish list. Have a fantastic holiday season!\n""","{""toy_list"": [""Star Wars LOLA animatronic droid"", ""Tiny Land kids fort building kit""], ""location"": ""Los Angeles""}",EN,968,0
2,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""\ncaller: Hello, I'm trying to submit a holiday wish list for my child but I'm not sure where to start.\nfrosty: Hello and welcome! I'd be happy to help. May I have your name please, and where are you calling from?\ncaller: My name is Laura, and I'm from Birmingham, Engla

#### Fine-tune Llama 2 Model

##### Configuration

We're passing `train_dataset` and `eval_dataset` that are used to generate loss calculation during fine-tuning process and we've set `output_weights_dir` as the directory where the fine-tuned weights will be stored after fine-tuning job completes. For better performance for the task, you will need at least 1 `num_epochs`, feel free to explore this on your own.

In [38]:
!mkdir merged_weights_dir
!mkdir output_weights_dir

mkdir: cannot create directory ‘merged_weights_dir’: File exists
mkdir: cannot create directory ‘output_weights_dir’: File exists


In [39]:
%%time
# NOTE: Fine-tuning using 100 records on this setup (compute and other resources) will take about ~5mins.

merged_weights_dir = 'merged_weights_dir'
output_weights_dir = 'output_weights_dir'

runner = ft_util.FinetuneModelRunner(
    base_model_id="meta-llama/Llama-2-7b-chat-hf",
    task_type=ft_util.FinetuneTaskType.INSTRUCT,
    train_dataset=train_df,
    eval_dataset=eval_df,
)

mcf = ft_util.ModelConfig(
    num_epochs=2,
    quantization=ft_util.QuantizationConfig(
        k_bits=4,
    ),
    merged_weights_dir_path=merged_weights_dir,
)

_ = runner.train(output_weights_dir, model_cfg=mcf)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


Map (num_proc=8):   0%|          | 0/1162 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/68 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
10,0.880400,0.801068
20,0.675600,0.347128
30,0.194100,0.137171
40,0.043300,0.110291
50,0.040900,0.097124
60,0.139000,0.086621
70,0.159900,0.083242
80,0.073700,0.061668
90,0.069500,0.060019
100,0.002600,0.061931


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO 11-15 19:00:31 llm_engine.py:72] Initializing an LLM engine with config: model='merged_weights_dir', tokenizer='merged_weights_dir', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 11-15 19:01:20 llm_engine.py:207] # GPU blocks: 864, # CPU blocks: 512
CPU times: user 40min 30s, sys: 30min 43s, total: 1h 11min 13s
Wall time: 1h 12min 11s


#### Evaluate Results on Eval Dataset

After the fine-tuning job completes, we call `get_eval_results`, this will run evaluation and get model predictions using the entire `eval_dataset`. You can see the output by model in the `predicted` column of the dataframe.

*Examine the "output" and "predicted" columns*

In [40]:
eval_df = runner.get_eval_results()
eval_df.head()

Processed prompts: 100%|██████████| 68/68 [00:14<00:00,  4.74it/s]


,instruction,input,output,language,id,lang_label,predicted
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi there! \nfrosty: Hello! Welcome to our Holiday Wish List line! How can I help you today? \ncaller: My name is Sarah and I'm calling from London. I wanted to submit a wish list for my child. \nfrosty: Wonderful, Sarah! It's great to have you on the line. What country do you live in? \ncaller: I live in the United Kingdom. \nfrosty: That's fantastic! What's on your wish list? \ncaller: I think my child would like the transformers bumblebee. \nfrosty: Great choice! Why are you interested in this toy? \ncaller: My child loves action figures and Transformers. I think he'll have a lot of fun with this one. \nfrosty: That sounds perfect! How do you and your family plan to celebrate the holiday season? \ncaller: We usually have a big family gathering at our house with lots of food and games. \nfrosty: That sounds lovely! Do you have a favorite holiday memory or pastime? \ncaller: My favorite memory is baking holiday cookies with my mom when I was a child. \nfrosty: That's a beautiful memory! So, just to confirm, you would like to add the Transformers Rise of the Beasts Beast-Mode Bumblebee to your wish list? \ncaller: Yes, that's correct! \nfrosty: Excellent! Thank you, Sarah, and I hope you and your family have a wonderful holiday season! \ncaller: Thank you! Have a great day! \nfrosty: You too! Goodbye!""","{""toy_list"": [""Transformers Rise of the Beasts Beast-Mode Bumblebee""], ""location"": ""London""}",EN,39,0,"{""toy_list"": [""Transformers Rise of the Beasts Beast-Mode Bumblebee""], ""location"": ""London""}"
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hello, my name is Shawn, and I'm calling from Vancouver. I need to submit a holiday wish list for my kids but I could use some help picking out the perfect toys.\nfrosty: Hi Shawn, from Vancouver! Let's find the best toys for your kids. Can you tell me a bit about their interests?\ncaller: My daughter is 5 years old, and she loves Sesame Street, especially Elmo. As for my son, he's 9 and really into Pok\u00e9mon.\nfrosty: I have just the toys in mind! How about the Sesame Street Monster Meditation Elmo for your daughter and the Pok\u00e9mon 8-inch Plush First Partner Three-Pack for your son?\ncaller: Those sound perfect! Please add them to the wish list.\nfrosty: I've added the Sesame Street Monster Meditation Elmo for your daughter and the Pok\u00e9mon 8-inch Plush First Partner Three-Pack for your son to the wish list. I hope they enjoy their gifts, and I wish your family a fantastic holiday season in Vancouver!""","{""toy_list"": [""Sesame Street Monster Meditation Elmo"", ""Pok\u00e9mon 8-Inch Plush First Partner Three-Pack""], ""location"": ""Vancouver""}",EN,525,0,"{""toy_list"": [""Sesame Street Monster Meditation Elmo"", ""Pok\u00e9mon 8-inch Plush First Partner Three-Pack""], ""location"": ""Vancouver""}"
2,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi, can you help me with my holiday wish list?\nfrosty: Of course! What's your name, and where are you calling from?\ncaller: My name is James, and I'm in Melbourne.\nfrosty: Hi James! Are you considering any specific toys from our catalog?\ncaller: I can't decide between the ninja turtles delivery van and the spiderman playset.\nfrosty: Both the Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van and Marvel's Spidey and His Amazing Friends Web Spinners playset are excellent choices! Why are you interested in these toys?\ncaller: My child loves superhero and action toys.\nfrosty: That makes sense! Can you tell me a l

#### Inference on Custom Transcript

Additionally, you may want to manually craft some cases to test the boundaries of the model. To do that, you can use the `infer` method to run prediction against a single sample interactively.

In [41]:
runner.infer(
     {
    'instruction': """
        "Extract JSON response with 'location' and 'toy_list' as keys. 'location': 
        Location of the caller. Include city only.'toy_list': List of toy names from the caller."
    """,
    'input': """
        "frosty: Hello, hello! Who's ready to chat with Frosty today?\ncaller: Hi, I'm Dustin from Cape Town, South Africa.\nfrosty: 
        Hi Dustin, it's awesome to meet you! Do you already know what you would like for your holiday gift?\ncaller: 
        Yeah, I think I want the ninja turtles delivery van, or maybe the robot dog. 
        I can't decide...\nfrosty: Both sound like entertaining choices! 
        Why are you interested in these particular toys?\ncaller: The ninja turtles van is so cool! And the robot dog, it's like a pet you don't have to clean up after.\nfrosty: 
        That's true! Savage choices, Dustin! How do you usually celebrate the holiday season?\ncaller: 
        We usually have a big family barbeque and play games all day long.\nfrosty: Sounds fun! What is your favorite thing about this time of year?
        \ncaller: I like that everyone is happy and I get to spend a lot of time with my family.
        \nfrosty: That's wonderful! So Dustin, have you thought any more about your holiday wish?
        \ncaller: I think I'm going to get the robot dog. It sounds fun and I won't have to walk it.
        \nfrosty: That's a clever choice, Dustin! I hope you have a fantastic holiday season!"
"""
    }
)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.02s/it]


' {"toy_list": ["Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van", "Hasbro\'s Tiny Pony Convertible and Learn-to-Ride Labyrinth"], "location": "Cape Town"}'

#### Batch Inference on Test Dataset

For batch inference to test on a holdout dataset, you can use `infer_batch` on a dataframe.

*Examine the "output" and "predicted" columns*

In [42]:
test_res = runner.infer_batch(test_df)
test_res.head()

Processed prompts: 100%|██████████| 138/138 [00:30<00:00,  4.57it/s]


,instruction,input,output,language,id,lang_label,predicted
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Bonjour ! Je suis Frosty, votre assistant pour les listes de souhaits des f\u00eates. Comment puis-je vous \u00eatre utile aujourd'hui ?\ncaller: Salut, je m'appelle Alain, j'ai besoin d'aide pour choisir un jouet pour mon enfant.\nfrosty: Bonjour Alain ! D'o\u00f9 appelez-vous ?\ncaller: Je vous appelle de Marseille.\nfrosty: Super ! Avez-vous une id\u00e9e du type de jouet que vous recherchez ?\ncaller: Je pensais au jurassic world dinosaur, mais je ne suis pas s\u00fbr.\nfrosty: Je comprends. Pourquoi \u00eates-vous int\u00e9ress\u00e9 par ce jouet ?\ncaller: Mon fils est un grand fan de dinosaures et de films Jurassic World.\nfrosty: C'est un bon point de d\u00e9part. Qu'est-ce que votre enfant appr\u00e9cie dans cette p\u00e9riode de l'ann\u00e9e ?\ncaller: Il aime recevoir des cadeaux et passer du temps avec sa famille.\nfrosty: C'est une p\u00e9riode id\u00e9ale pour des moments en famille. Avant de d\u00e9cider, parlons d'autres options de cadeaux. Que pensez-vous du snap circuit set pour un enfant qui aime les dinosaures et les films ?\ncaller: Je pense que ce pourrait \u00eatre int\u00e9ressant, mais mon fils pr\u00e9f\u00e9rerait peut-\u00eatre le jurassic world dinosaur.\nfrosty: D'accord, dans ce cas, ajoutons le jurassic world dinosaur \u00e0 votre liste de souhaits. Est-ce qu'il y a un autre jouet que vous aimeriez ajouter ?\ncaller: Non, c'est tout pour le moment. Merci pour votre aide, Frosty.\nfrosty: C'est un plaisir de vous aider, Alain ! Passez d'excellentes vacances en famille. Au revoir !""","{""toy_list"": [""Jurassic World Dominion Super Colossal Giganotosaurus""], ""location"": ""Marseille""}",FR,1173,1,"{""toy_list"": [""Jurassic World Dominion Super Colossal Giganotosaurus""], ""location"": ""Marseille""}"
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi Frosty, my name is Daniel, and I'm calling from Cape Town. Could you help me finalize my child's holiday wish list?\nfrosty: Of course, Daniel from Cape Town! Let's find the perfect toy for your child. What type of toys are they interested in?\ncaller: He's really into dinosaurs and anything related to them. I was thinking of getting him the jurassic world dinosaur.\nfrosty: The Jurassic World Dominion Super Colossal Giganotosaurus is an excellent choice for a dinosaur enthusiast! Can you tell me what you enjoy the most about this time of the year?\ncaller: What I love the most is the atmosphere of joy and sharing special moments with family and friends.\nfrosty: That's a lovely answer, Daniel. I'm adding the Jurassic World Dominion Super Colossal Giganotosaurus to your child's holiday wish list. Have a fantastic holiday season!""","{""toy_list"": [""Jurassic World Dominion Super Colossal Giganotosaurus""], ""location"": ""Cape Town""}",EN,938,0,"{""toy_list"": [""Jurassic World Dominion Super Colossal Giganotosaurus""], ""location"": ""Cape Town""}"
2,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: \u00a1Hola y bienvenido! Soy Frosty, tu asistente de voz para la lista de deseos de vacaciones. \u00bfCu\u00e1l es tu nombre y desde d\u00f3nde est\u00e1s llamando?\n\ncaller: \u00a1Hola Frosty! Soy Andr\u00e9s. Estoy llamando desde Santiago, Chile.\n\nfrosty: Hola, Andr\u00e9s. Encantado de conocerte. \u00bfQu\u00e9 juguetes te gustar\u00eda agregar a la lista de deseos para tus hijos esta temporada?\n\ncaller: No estoy seguro todav\u00eda. Me gustar\u00eda explorar el cat\u00e1logo de juguetes para encontrar el perfecto para mi hijo.\n\nfrosty: Claro, con gusto te ayudo.

### Log and Deploy Fine-tuned Llama 2

In [43]:
MODEL_NAME = "LLAMA2_7b_CHAT"
MODEL_VERSION = "BestTunedV1.0"
DEPLOYMENT_NAME = "BESTTUNED_LLAMA2"
MODEL_REGISTRY_DB = connection_parameters['database']
MODEL_REGISTRY_SCHEMA = connection_parameters['schema']
COMPUTE_POOL = connection_parameters['compute_pool']

registry = model_registry.ModelRegistry(
    session=session, 
    database_name=MODEL_REGISTRY_DB, 
    schema_name=MODEL_REGISTRY_SCHEMA, 
    create_if_not_exists=True)

#### Delete any existing deployments and models from in Snowpark registry to avoid resource contention during this bootcamp

In [44]:
registry.list_deployments(model_name=MODEL_NAME,model_version=MODEL_VERSION).to_pandas()

,MODEL_NAME,MODEL_VERSION,DEPLOYMENT_NAME,CREATION_TIME,TARGET_METHOD,TARGET_PLATFORM,SIGNATURE,OPTIONS,STAGE_PATH,ROLE


In [45]:
# registry.delete_deployment(model_name=MODEL_NAME,model_version=MODEL_VERSION,deployment_name=DEPLOYMENT_NAME)

In [46]:
# registry.delete_model(model_name=MODEL_NAME,model_version=MODEL_VERSION,delete_artifact=True)

### Log and Deploy Fine-tuned Llama 2

*Note: Logging and deploying fine-tuned model on this setup (compute and other resources) will take about ~15mins*

Notice that the "model_id_or_path" is set to the output weights folder `output_weights_dir` where the weights from fine-tuning the model are stored

In [47]:
%%time
options = llm.LLMOptions(
    token=connection_parameters["huggingface_token"],
    max_batch_size=100,
)

llama_model = llm.LLM(
    model_id_or_path=output_weights_dir,
    options=options
)

llama_model_ref = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=llama_model
)

llama_model_ref.deploy(
    deployment_name=DEPLOYMENT_NAME, 
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True, 
    options={"compute_pool": COMPUTE_POOL, "num_gpus": 1})

/opt/conda/envs/rapids/lib/python3.8/site-packages/snowflake/ml/model/_packager/model_env/model_env.py:353: UserWarning: Found dependencies specified as pip requirements. This may prevent model deploying to Snowflake Warehouse.
  warnings.warn(


CPU times: user 5.52 s, sys: 660 ms, total: 6.18 s
Wall time: 17min 59s


{'name': 'DASH_DB.DASH_SCHEMA.BESTTUNED_LLAMA2',
 'platform': <TargetPlatform.SNOWPARK_CONTAINER_SERVICES: 'SNOWPARK_CONTAINER_SERVICES'>,
 'target_method': 'infer',
 'signature': ModelSignature(
                     inputs=[
                         FeatureSpec(dtype=DataType.STRING, name='input')
                     ],
                     outputs=[
                         FeatureSpec(dtype=DataType.STRING, name='generated_text')
                     ]
                 ),
 'options': {'compute_pool': 'DASH_BUILD_GPU3', 'num_gpus': 1},
 'details': {'image_name': 'sfsenorthamerica-build-spcs.registry.snowflakecomputing.com/dash_db/dash_schema/snowml_repo/a2dc5950178b8bf1c3764f03dddcbe86ea99c226:latest',
  'service_spec': "spec:\n  container:\n  - env:\n      MODEL_ZIP_STAGE_PATH: /DASH_DB.DASH_SCHEMA.SNOWML_MODEL_D26762D883E911EE8F5ACE7DB0E7935F/model.zip\n      NUM_WORKERS: 1\n      SNOWML_USE_GPU: true\n      TARGET_METHOD: infer\n      _CONCURRENT_REQUESTS_MAX: 1\n    image: sfsen

#### Prepare Eval Dataset with Specific Instructions

In [48]:
begin_prompt = \
"""
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
Extract JSON response with 'location' and 'toy_list' as keys.
'location': Location string of the caller.
'toy_list': List of toy names from the caller.
### Input:

"""
end_prompt = " [/INST]"

eval_df['input'] = begin_prompt + eval_df['input'] + end_prompt
eval_df = eval_df.reset_index(drop=True)
eval_df.head()

,instruction,input,output,language,id,lang_label,predicted
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"\n[INST] <<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n### Instruction:\nExtract JSON response with 'location' and 'toy_list' as keys.\n'location': Location string of the caller.\n'toy_list': List of toy names from the caller.\n### Input:\n\n""caller: Hi there! \nfrosty: Hello! Welcome to our Holiday Wish List line! How can I help you today? \ncaller: My name is Sarah and I'm calling from London. I wanted to submit a wish list for my child. \nfrosty: Wonderful, Sarah! It's great to have you on the line. What country do you live in? \ncaller: I live in the United Kingdom. \nfrosty: That's fantastic! What's on your wish list? \ncaller: I think my child would like the transformers bumblebee. \nfrosty: Great choice! Why are you interested in this toy? \ncaller: My child loves action figures and Transformers. I think he'll have a lot of fun with this one. \nfrosty: That sounds perfect! How do you and your family plan to celebrate the holiday season? \ncaller: We usually have a big family gathering at our house with lots of food and games. \nfrosty: That sounds lovely! Do you have a favorite holiday memory or pastime? \ncaller: My favorite memory is baking holiday cookies with my mom when I was a child. \nfrosty: That's a beautiful memory! So, just to confirm, you would like to add the Transformers Rise of the Beasts Beast-Mode Bumblebee to your wish list? \ncaller: Yes, that's correct! \nfrosty: Excellent! Thank you, Sarah, and I hope you and your family have a wonderful holiday season! \ncaller: Thank you! Have a great day! \nfrosty: You too! Goodbye!"" [/INST]","{""toy_list"": [""Transformers Rise of the Beasts Beast-Mode Bumblebee""], ""location"": ""London""}",EN,39,0,"{""toy_list"": [""Transformers Rise of the Beasts Beast-Mode Bumblebee""], ""location"": ""London""}"
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"\n[INST] <<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n### Instruction:\nExtract JSON response with 'location' and 'toy_list' as keys.\n'location': Location string of the caller.\n'toy_list': List of toy names from the caller.\n### Input:\n\n""caller: Hello, my name is Shawn, and I'm calling from Vancouver. I need to submit a holiday wish list for my kids but I could use some help picking out the perfect toys.\nfrosty: Hi Shawn, from Vancouver! Let's find the best toys for your kids. Can you tell me a bit about their interests?\ncaller: My daughter is 5 years old, and she loves Sesame Street, especially Elmo. As for my son, he's 9 and really into Pok\u00e9mon.\nfrosty: I have just the toys in mind! How about the Sesame Street Monster Meditation Elmo for your daughter and the Pok\u00e9mon 8-inch Plush First Partner Three-Pack for your son?\ncaller: Those sound perfect! Please add them to the wish list.\nfrosty: I've added the Sesame Street Monster Meditation Elmo for your daughter and the Pok\u00e9mon 8-inch Plush First Partner Three-Pack for your son to the wish list. I hope they enjoy their gifts, and I wish your family a fantastic holiday season in Vancouver!"" [/INST]","{""toy_list"": [""Sesame Street Monster Meditation Elmo"", ""Pok\u00e9mon 8-Inch Plush First Partner Three-Pack""], ""location"": ""Vancouver""}",EN,525,0,"{""toy_list"": [""Sesame Street Monster Meditation Elmo"", ""Pok\u00e9mon 8-inch Plush First Partner Three-Pack""], ""location"": ""Vancouver""}"
2,Extract JSON response with 'location' and 'toy_lis

#### Inference on Eval Dataset using fine-tuned Llama 2

In [49]:
llama_model_ref.predict(deployment_name=DEPLOYMENT_NAME,data=eval_df).head()

,generated_text
0,"{""toy_list"": [""Transformers Rise of the Beasts Beast-Mode Bumblebee""], ""location"": ""London""}"
1,"{""toy_list"": [""Sesame Street Monster Meditation Elmo"", ""Pok\u00e9mon 8-inch Plush First Partner Three-Pack""], ""location"": ""Vancouver""}"
2,"{""toy_list"": [""Marvel's Spidey and His Amazing Friends Web Spinners playset""], ""location"": ""Melbourne""}"
3,"{""toy_list"": [""LeapFrog Magic Adventures Microscope""], ""location"": ""Chicago""}"
4,"{""toy_list"": [""Furby interactive plush toy""], ""location"": ""Wellington""}"


#### Run Inference in SQL

In [ ]:
# TODO: Go to Snowsight and run the following commands in a SQL Worksheet

# create or replace function build_prompt(inp varchar)
# returns varchar
# language sql
# as $$
#     '[INST] <<SYS>>\n' ||
#     'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n' ||
#     '<</SYS>>\n' ||
#     '### Instruction:\n' ||
#     'Extract JSON response with \'location\' and \'toy_list\' as keys.\n' ||
#     '\'location\': Location of the caller. Include city only.\n' ||
#     '\'toy_list\': List of toy names from the caller.\n' ||
#     '### Input:\n' ||
#     '"' || inp || '"\n' ||
#     '[/INST]'
# $$;

# SELECT TRAINED_LLAMA2(object_construct('input', build_prompt(select transcript from frosty_transcripts limit 1)));

#### Clean Up Resources

Delete deployment and the model

In [ ]:
llama_model_ref.delete_deployment(deployment_name=DEPLOYMENT_NAME)
llama_model_ref.delete_model(delete_artifact=True)